# Importing the needed libraries and files

In [ ]:
import openai,files,fitz
from PyPDF2 import PdfReader, PdfWriter

In [ ]:
openai.api_key = files.openaikey

opening with fitz

In [ ]:
doc = fitz.open(files.bnk)
doc.authenticate(files.password)
new=fitz.open()
page_number = 0  
text = doc[page_number].get_text()

authenticating... with pypdf2

In [ ]:
reader = PdfReader(files.bnk)
writer = PdfWriter()

if reader.is_encrypted:
    reader.decrypt(files.password)

for page in reader.pages:
    writer.add_page(page)

# Save the new PDF to a file
with open("decrypted-pdf.pdf", "wb") as f:
    writer.write(f)

Extracting the confidential data from the text

In [ ]:
page = reader.pages[0]
extracted_text = page.extract_text()
# print(extracted_text)
original_entities = {}
for line in extracted_text.split('\n'):
    if ':' in line:
        try:
            key, value = line.split(':')
            original_entities[key.strip()] = value.strip()
        except ValueError:
            pass


In [ ]:
original_entities

Creating a dictionary to store original data and synthetic data correspondence


In [ ]:
correspondence_dict = {}

generating synthetic data

In [ ]:
for entity_type,entity in original_entities.items():
    prompt1 = f"Generate a synthetic replacement for the {entity_type.lower()}: '{entity}'."
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt1,
        max_tokens=20
    )
    synthetic_data = response.choices[0].text.strip()
    correspondence_dict[entity] = synthetic_data

In [ ]:
correspondence_dict

Masking the original text information

In [ ]:
masked_text = text


In [ ]:
masked_text

In [ ]:
for org_data, syn_data in correspondence_dict.items():
    masked_text = masked_text.replace(org_data, syn_data)

saving to pdf

In [ ]:
modified_image = doc[page_number].get_pixmap(matrix=fitz.Matrix(1, 1), clip=doc[page_number].rect, alpha=False)
new_page = new.new_page(width=doc[page_number].rect.width, height=doc[page_number].rect.height)
new_page.insert_text((10, 10),masked_text)
new.save('modified_document.pdf')

In [ ]:
doc.close()
new.close()

# Unmasking

In [ ]:
doc = fitz.open('modified_document.pdf')
# doc.authenticate(files.password)
new=fitz.open()
page_number = 0  
text = doc[page_number].get_text()

In [ ]:
unmasked_text = text
for org_data, syn_data in correspondence_dict.items():
   unmasked_text = unmasked_text.replace(syn_data,org_data)

In [ ]:
modified_image = doc[page_number].get_pixmap(matrix=fitz.Matrix(1, 1), clip=doc[page_number].rect, alpha=False)
new_page = new.new_page(width=doc[page_number].rect.width, height=doc[page_number].rect.height)
new_page.insert_text((10, 10),unmasked_text, fontname="helv", fontsize=12)
new.save('unmasked.pdf')